In [5]:
! pip install tabpfn
! pip install feature_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/AIGO-2025

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/AIGO-2025


In [33]:
import os
import pandas as pd
from tabpfn import TabPFNClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import f1_score
from sklearn.base import TransformerMixin
import lightgbm as lgb
from feature_engine.outliers import Winsorizer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter

# Step 1: Read Data

In [4]:
df_train = pd.read_csv("data/training.csv")
df_test = pd.read_csv("data/public_x.csv")

In [7]:
print(df_train.shape)
print(df_test.shape)

(200864, 10214)
(25108, 10213)


In [ ]:
df_train.head()

,ID,外資券商_分點進出,外資券商_分點買賣力,外資券商_分點成交力(%),外資券商_分點吃貨比(%),外資券商_分點出貨比(%),外資券商_前1天分點進出,外資券商_前1天分點買賣力,外資券商_前1天分點成交力(%),外資券商_前1天分點吃貨比(%),...,上市加權指數前15天成交量,上市加權指數前16天成交量,上市加權指數前17天成交量,上市加權指數前18天成交量,上市加權指數前19天成交量,上市加權指數前20天成交量,上市加權指數5天成交量波動度,上市加權指數10天成交量波動度,上市加權指數20天成交量波動度,飆股
0,TR-1,1.2227,2.8303,1.2789,0.5707,0.5329,1.2219,1.2686,1.2255,0.5228,...,0.3121,0.1312,0.2415,0.3157,0.4439,0.0533,0.1951,0.1357,0.2388,0
1,TR-2,1.2297,2.8303,1.7995,1.1170,0.5329,1.2313,1.4432,1.2657,1.0660,...,0.9439,0.7407,0.4615,0.4663,0.6618,0.1356,0.4063,0.5800,0.3199,0
2,TR-3,1.3127,1.2826,1.4307,5.0469,4.7141,1.1049,1.2271,1.2514,3.0514,...,0.4925,0.7805,1.2576,0.9081,1.1555,1.4803,0.1924,0.4615,0.7695,0
3,TR-4,1.2219,NaN,1.2413,0.5313,0.5329,1.2258,NaN,1.2238,0.5228,...,1.7776,0.8460,0.8016,0.8309,1.6546,1.5733,0.7886,0.2080,0.7321,0
4,TR-5,0.7988,1.0454,0.8652,1.8842,2.2257,0.7108,1.1333,1.1712,1.5242,...,0.7412,0.8641,0.8693,0.7486,0.2326,0.4883,1.8395,1.4787,0.9049,0


In [ ]:
df_train['飆股'].value_counts()

,count
飆股,
0,199394
1,1470


In [ ]:
cols = df_train.columns
for col in df_train.columns:
    if '代號' in col and '買賣超' in col:
      print(col)

# Step 2: Feature Engineering

### Select feature

In [ ]:
# import pandas as pd
# import random

# random.seed(123)
# ranking_df = pd.read_csv('./eda_output_polars_v12/feature_ic_original_ranking_full subset.csv')

# top_pct = 0.1
# top_n = int(len(ranking_df) * top_pct)

# # top_features = ranking_df['Feature'].iloc[:top_n].tolist()
# top_features = random.sample(ranking_df['Feature'].tolist(), top_n)
# print(len(top_features))


803


In [8]:
keywords = [
    '外資', '投信', '自營商', '賣賣超', # 籌碼
    '乖離率', '收盤價', '技術指標_K', '技術指標_D', 'MACD',  # 技術
    '月營收', '營業利益', '毛利率' # 基本
    ]
selected_cols = [col for col in df_train.columns if any(kw in col for kw in keywords)]

print(len(selected_cols))

269


In [9]:
df_train_selected = df_train[selected_cols]
df_test_selected = df_test[selected_cols]

### Delete all empty

In [10]:
all_nan_cols = [col for col in df_train_selected.columns if df_train_selected[col].isna().all()]

df_train_selected = df_train_selected.drop(columns=all_nan_cols)
df_test_selected = df_test_selected.drop(columns=all_nan_cols)

print(df_train_selected.shape)
print(df_test_selected.shape)

(200864, 262)
(25108, 262)


### Imputation

In [11]:
# All numerical fields are processed
num_cols = df_train_selected.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [12]:
imputer = SimpleImputer(strategy='mean')
df_train_selected[num_cols] = pd.DataFrame(
    imputer.fit_transform(df_train_selected[num_cols]),
    columns=num_cols,
    index=df_train_selected.index
)
df_test_selected[num_cols] = pd.DataFrame(
    imputer.transform(df_test_selected[num_cols]),
    columns=num_cols,
    index=df_test_selected.index
)

### Outlier Handling

In [13]:
# trim the two tails.
# winsor = Winsorizer(capping_method='iqr', tail='both', fold=1.5, variables=num_cols) # some columns have low variation for method 'iqr'.
winsor = Winsorizer(capping_method='gaussian', tail='both', fold=1.5, variables=num_cols)

df_train_selected = winsor.fit_transform(df_train_selected)
df_test_selected = winsor.transform(df_test_selected)

In [14]:
df_train_final = pd.concat([df_train[['ID']], df_train_selected, df_train[['飆股']]], axis=1)
df_test_final = pd.concat([df_test[['ID']], df_test_selected], axis=1)

In [15]:
print(df_train_final.shape)
print(df_test_final.shape)

(200864, 264)
(25108, 263)


In [16]:
df_train_final.columns

Index(['ID', '外資券商_分點進出', '外資券商_分點買賣力', '外資券商_分點成交力(%)', '外資券商_分點吃貨比(%)',
       '外資券商_分點出貨比(%)', '外資券商_前1天分點進出', '外資券商_前1天分點買賣力', '外資券商_前1天分點成交力(%)',
       '外資券商_前1天分點吃貨比(%)',
       ...
       '上市加權指數前15天收盤價', '上市加權指數前16天收盤價', '上市加權指數前17天收盤價', '上市加權指數前18天收盤價',
       '上市加權指數前19天收盤價', '上市加權指數前20天收盤價', '上市加權指數5天乖離率', '上市加權指數10天乖離率',
       '上市加權指數19天乖離率', '飆股'],
      dtype='object', length=264)

### train_test_split

In [46]:
X = df_train_final.drop(columns=['ID', '飆股'])
y = df_train_final['飆股']

# Use stratify to maintain proportions in all categories
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [47]:
X_train.values

array([[ 1.2219    ,  1.22143161,  1.2413    , ...,  1.1101    ,
         1.2089    ,  1.3361    ],
       [ 1.0993    ,  0.6022    , -0.26661657, ...,  1.048     ,
         0.5021    ,  0.974     ],
       [ 1.2343    ,  2.47491662,  1.9059    , ...,  0.8849    ,
         0.7694    ,  1.394     ],
       ...,
       [ 1.4897    ,  1.3103    ,  1.3937    , ..., -0.1233    ,
        -0.2791    , -0.27877035],
       [ 1.1769    ,  0.5406    ,  0.2866    , ...,  1.4821    ,
         0.3922    ,  0.0277    ],
       [ 1.246     ,  2.4441    ,  1.4899    , ...,  2.72251835,
         2.72215802,  2.72235353]])

In [48]:
X_train

,外資券商_分點進出,外資券商_分點買賣力,外資券商_分點成交力(%),外資券商_分點吃貨比(%),外資券商_分點出貨比(%),外資券商_前1天分點進出,外資券商_前1天分點買賣力,外資券商_前1天分點成交力(%),外資券商_前1天分點吃貨比(%),外資券商_前1天分點出貨比(%),...,上市加權指數前14天收盤價,上市加權指數前15天收盤價,上市加權指數前16天收盤價,上市加權指數前17天收盤價,上市加權指數前18天收盤價,上市加權指數前19天收盤價,上市加權指數前20天收盤價,上市加權指數5天乖離率,上市加權指數10天乖離率,上市加權指數19天乖離率
186563,1.2219,1.221432,1.241300,0.531300,0.532900,1.2258,1.221389,1.2238,0.52280,0.534300,...,0.072600,0.084000,0.088400,0.092000,0.077300,0.089100,0.077600,1.110100,1.208900,1.336100
5109,1.0993,0.602200,-0.266617,1.942400,2.713048,1.1787,1.047800,0.9974,2.71425,2.715211,...,-0.281451,-0.281491,-0.281555,-0.281497,-0.281541,-0.281476,-0.281591,1.048000,0.502100,0.974000
171499,1.2343,2.474917,1.905900,1.315900,0.617300,1.2133,1.513900,1.2535,0.52280,0.909900,...,1.748200,1.752400,1.711300,1.678700,1.676500,1.704600,1.689000,0.884900,0.769400,1.394000
90205,0.5528,0.113400,-0.014600,0.792700,2.062800,-0.1755,1.128300,1.1653,1.20870,2.715211,...,1.909800,1.904400,1.896400,1.981300,1.962400,2.008900,2.032000,2.722518,2.497200,2.082300
79364,1.2227,1.305600,1.253800,0.714400,0.697600,1.2156,1.218600,1.2259,0.71500,1.069700,...,-0.073100,-0.105600,-0.115700,-0.107000,-0.111600,-0.136400,-0.141400,0.909200,0.776900,1.329600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6793,1.2273,2.474917,1.552000,0.857200,0.532900,1.2290,1.744000,1.2451,0.68030,0.534300,...,1.521600,1.522300,1.564400,1.553200,1.510500,1.479000,1.427900,2.247000,2.267400,1.963600
149730,1.2592,1.992000,2.709296,2.712229,1.686200,1.2133,1.733000,1.4327,0.86110,1.743300,...,2.444100,2.403200,2.425700,2.520500,2.621500,2.634700,2.646900,2.520100,2.722158,0.896800
107861,1.4897,1.310300,1.393700,2.611900,2.393200,0.2955,1.218900,1.2360,2.33760,2.715211,...,1.671400,1.625400,1.645600,1.677800,1.753500,1.761000,1.776800,-0.123300,-0.279100,-0.278770
137641,1.1769,0.540600,0.286600,1.153200,2.105700,1.1999,0.924900,1.1263,0.84920,1.552400,...,1.761000,1.781500,1.730100,1.671300,1.611200,1.696300,1.699700,1.482100,0.392200,0.027700


In [49]:
class ShapeLogger(TransformerMixin):
    def __init__(self, name=""):
        self.name = name

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        print(f"Shape after {self.name}: {X.shape}")
        return X

In [50]:
target_count = 10000
undersample_strategy = {0: target_count, 1: y_train.value_counts()[1]}
print("Before undersampling:", Counter(y_train))

pipeline = Pipeline(steps=[
    ('under', RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=42)),
    ('log1', ShapeLogger(name="RandomUnderSampler")),
    ('smote', SMOTE(k_neighbors=5, random_state=42, sampling_strategy='auto')),
    ('log2', ShapeLogger(name="SMOTE")),
    ('feature_selection', SelectFromModel(lgb.LGBMClassifier(random_state=42), threshold='median')),
    ('log3', ShapeLogger(name="Feature Selection")),
    ('classifier', TabPFNClassifier(ignore_pretraining_limits=True))
])

Before undersampling: Counter({0: 159515, 1: 1176})


# Step 3: Train

In [51]:
pipeline.fit(X_train, y_train)

Shape after RandomUnderSampler: (11176, 262)
Shape after SMOTE: (20000, 262)
[LightGBM] [Info] Number of positive: 10000, number of negative: 10000
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014183 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 66376
[LightGBM] [Info] Number of data points in the train set: 20000, number of used features: 262
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Shape after Feature Selection: (20000, 136)


/usr/local/lib/python3.11/dist-packages/tabpfn/classifier.py:421: UserWarning: Number of samples 20000 is greater than the maximum Number of samples 10000 supported by the model. You may see degraded performance.
  X, y, feature_names_in, n_features_in = validate_Xy_fit(


Pipeline(steps=[('under',
                 RandomUnderSampler(random_state=42,
                                    sampling_strategy={0: 10000,
                                                       1: np.int64(1176)})),
                ('log1', <__main__.ShapeLogger object at 0x7ff1855df310>),
                ('smote', SMOTE(random_state=42)),
                ('log2', <__main__.ShapeLogger object at 0x7ff07f212590>),
                ('feature_selection',
                 SelectFromModel(estimator=LGBMClassifier(random_state=42),
                                 threshold='median')),
                ('log3', <__main__.ShapeLogger object at 0x7ff07f211610>),
                ('classifier',
                 TabPFNClassifier(ignore_pretraining_limits=True))])

# Step 4: evaluate

In [57]:
# transform X_val through all the steps in the Pipeline (except for the classifier).
X_val_transformed = pipeline[:-1].transform(X_val)
probas = pipeline.named_steps['classifier'].predict_proba(X_val_transformed)
y_pred = (probas[:, 1] > 0.5).astype(int)

Shape after RandomUnderSampler: (40173, 262)
Shape after SMOTE: (40173, 262)
Shape after Feature Selection: (40173, 136)


/usr/local/lib/python3.11/dist-packages/imblearn/pipeline.py:65: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 0.15 instead of the current warning.
  warnings.warn(


In [58]:
acc = accuracy_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print("Accuracy: ", acc)
print("Confusion Matrix:\n", cm)
print("F1 Score:", f1)

Accuracy:  0.9749334129888233
Confusion Matrix:
 [[38944   935]
 [   72   222]]
F1 Score: 0.3059958649207443


### Step 5: Test

In [60]:
X_test = df_test_final.drop(columns=['ID'])

X_test_transformed = pipeline[:-1].transform(X_test)
probas = pipeline.named_steps['classifier'].predict_proba(X_test_transformed)
y_test_pred = (probas[:, 1] > 0.5).astype(int)

Shape after RandomUnderSampler: (25108, 262)
Shape after SMOTE: (25108, 262)
Shape after Feature Selection: (25108, 136)


/usr/local/lib/python3.11/dist-packages/imblearn/pipeline.py:65: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 0.15 instead of the current warning.
  warnings.warn(


In [62]:
submission = pd.DataFrame({
    'ID': df_test_final['ID'],
    '飆股': y_test_pred
})
os.makedirs("output", exist_ok=True)
submission.to_csv("output/submission_TabPFN_202504041500.csv", index=False)

submission

,ID,飆股
0,PU-1,0
1,PU-2,0
2,PU-3,0
3,PU-4,0
4,PU-5,0
...,...,...
25103,PU-25104,0
25104,PU-25105,0
25105,PU-25106,0
25106,PU-25107,0
